# Exploring the MIMIC-III Patient Database
The MIMIC-III database contains several files. To limit our scope to the factors identified by the SIRS criteria, we will focus on:
- ADMISSIONS.csv: To get the unique hospital admission ID (HADM_ID) and patient unique identifier (SUBJECT_ID)
- PATIENTS.csv: To get the patient's gender and DOB (to calculate age of admission)
- CHARTS.csv: To get the vitals and white blood count reports

In [ ]:
# If environment does not support pandas, uncomment and install
#!pip install pandas

In [ ]:
import os
import numpy as np
import pandas as pd

## Exploring the Admission data

In [ ]:
import gzip
zip_file='mimic-iii-clinical-database-1.4/ADMISSIONS.csv.gz'
f = gzip.open(zip_file, 'rb')
#file_content = f.read()
#f.close()

#admission_file='mimic-iii-clinical-database-1.4/ADMISSIONS.csv'
#df_admission = pd.read_csv(admission_file)
df_admission = pd.read_csv(f)

In [2]:
data_dir = 'MIMIC-data'
df_admission = pd.read_csv(os.path.join(data_dir,'ADMISSIONS.csv'))

In [3]:
df_admission.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


### Useful columns in Admissions table:
- SUBJECT_ID
- HADM_ID
- ADMITTIME
- ADMISSION_TYPE
- DIAGNOSIS

In [4]:
df_admission['SUBJECT_ID'].nunique()

46520

In [5]:
df_admission['HADM_ID'].nunique()

58976

In [6]:
df_admission.groupby('ADMISSION_TYPE')['HADM_ID'].count()

ADMISSION_TYPE
ELECTIVE      7706
EMERGENCY    42071
NEWBORN       7863
URGENT        1336
Name: HADM_ID, dtype: int64

### The dataset contains:
- More than 40,000 unique patients
- More than 50,000 hospital admissions
- 4 admission types

## Extract sepsis flag from DIAGNOSIS

In [7]:
# Flag hospital admissions that has sepsis in the diagnosis
df_admission['SEPSIS_FLAG'] = df_admission['DIAGNOSIS'].str.contains('SEPSIS')

In [8]:
df_admission.groupby('ADMISSION_TYPE')['SEPSIS_FLAG'].value_counts()

ADMISSION_TYPE  SEPSIS_FLAG
ELECTIVE        False           7692
                True               1
EMERGENCY       False          40317
                True            1745
NEWBORN         False           7863
URGENT          False           1296
                True              37
Name: SEPSIS_FLAG, dtype: int64

In [9]:
df_admission.groupby('ADMISSION_TYPE')['SEPSIS_FLAG'].count()

ADMISSION_TYPE
ELECTIVE      7693
EMERGENCY    42062
NEWBORN       7863
URGENT        1333
Name: SEPSIS_FLAG, dtype: int64

## Reduce rows
Since 'ELECTIVE' and 'NEW_BORN' contain few SEPSIS case, keep the rows for "EMERGENCY" and "URGENT" admission types only.

In [10]:
df_admission = df_admission[(df_admission['ADMISSION_TYPE']=='EMERGENCY') |(df_admission['ADMISSION_TYPE']=='URGENT')]

In [11]:
df_admission.groupby('ADMISSION_TYPE')['HADM_ID'].count()

ADMISSION_TYPE
EMERGENCY    42071
URGENT        1336
Name: HADM_ID, dtype: int64

### Keep columns: 
'SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','ADMISSION_TYPE','SEPSIS_FLAG'


In [12]:
df_admission = df_admission[['SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','ADMISSION_TYPE','SEPSIS_FLAG']]
df_admission.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,SEPSIS_FLAG
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,EMERGENCY,False
2,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,EMERGENCY,False
3,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,EMERGENCY,False
4,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,EMERGENCY,False
5,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,EMERGENCY,False


### Write trimmed admission records to csv

In [13]:
df_admission.to_csv(os.path.join(data_dir,'ADMISSIONS_TRIM.csv'), index=None)

## Exploring patients data

In [14]:
df_patients = pd.read_csv(os.path.join(data_dir,'PATIENTS.csv'))

In [15]:
df_patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46520 entries, 0 to 46519
Data columns (total 8 columns):
ROW_ID         46520 non-null int64
SUBJECT_ID     46520 non-null int64
GENDER         46520 non-null object
DOB            46520 non-null object
DOD            15759 non-null object
DOD_HOSP       9974 non-null object
DOD_SSN        13378 non-null object
EXPIRE_FLAG    46520 non-null int64
dtypes: int64(3), object(5)
memory usage: 2.8+ MB


In [16]:
import matplotlib.pyplot as plt 
%matplotlib inline

In [17]:
df_patients.SUBJECT_ID.nunique()

46520

### Observations
There are 46,520 unique patients in the patient database -- more than half are male patients.<br>
The useful columns for analysis are SUBJECT_ID, GENDER, DOB

### Next Step
The CHARTEVENTS.csv is a very large file, and will need to separately processed in the next notebook before analysis.